Basics:

CPU 
Storage - A,B numbers are in storage
Numbers read from storage into registers in cpu
Then variables multiplied and stored in third register
Third is written into storage

If sequential:
step latency - time it takes to finish 1 step
total latency - time to finish all steps
Main memory is fast
spinning disk is slow

Most latency is from memory latency, not operational latency
Most latency is from reading and writing

Sorting can improve memory locality
Reason is money - local memory faster
CPU checks if memory location in cache, and if it's not there,
then retrieval will be slow. First case is a cache hit 
(when CPU intends to read a location and it's in cache)
Second case is a cache miss.

To handle a cache miss: 
(1) Free space in cache
(2) After freeing, read in memory location and copy the block size
Caches are effective if they have high hit rates.

Unsorted word counts would entail temporal locality for common words, but
no spatial locality.

Caching reduces latency by bringing relevant data closer to the CPU.

Access locality is the ability of software to make good use of the cache

Temporal Locality = locality accessing the same elemnt over and over again

Spatial Locality - ways to store n x terms to be squared. 
Linked lists have poor locality whereas indexed-arrays have
good locality.

Arrays store elements consecutively

Row by row for array scanning is faster

Effect increases proportionally with the number of elements in array


Memory Hierarchy from small/fast storage closest to CPU at top.

CPU (registers) - L2 Cache - Memory - Disk

CPU registers -> L1/L2/L3 cache -> RAM -> SSDs -> Magnetic Tapes

Data Centers are the physical aspect of "the cloud"


HDFS - each file is broken into fixed-size chunks that are then copied.
* Low cost per byte of storage 
* Locality
* Redundancy - can recover from server failures
* simple abstraction - looks like standard file system


In [3]:
sc

res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@5649d52a


In [4]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession


In [5]:
val spark = SparkSession
  .builder
  .appName("portdata-causality")
  .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3b61558f


In [6]:
import spark.implicits._

import spark.implicits._


In [7]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [8]:
spark.conf.set("spark.sql.retainGroupColumns","false")

In [9]:
%%python
# import pandas as pd
# probeDf = pd.read_csv("sparkProbeData.csv")
# sparkProbeDf = spark.createDataFrame(probeDf)
# sparkProbeDf = sparkProbeDf.drop("Unnamed: 0")
# probeLabeledDf = pd.read_csv("sparkProbeLabeledData.csv")
# probeLabeledDf.head()
# sparkProbeLabeledDf = spark.createDataFrame(probeLabeledDf)
# sparkProbeLabeledDf = sparkProbeLabeledDf.drop("Unnamed: 0")

IndexError: ('pop from empty list',)

In [ ]:
/* 
ip_flags, tcp_flags_ack, ip_dsfield, tcp_seq, tcp_flags_fin, tcp_flags_urg,tcp_flags_push
*/

In [10]:
%%python
probeDf = spark.read.csv("newPortData.csv",header=True)

In [11]:
%%python
# "ip_flags", "tcp_flags_ack",\
#                 "ip_dsfield",\
#                 "tcp_seq", "tcp_flags_fin",\
#                 "tcp_flags_urg",\
#                 "tcp_flags_push",\
#                 "tcp_options_mss_val",\
#                 "ip_ttl",\
#                 "tcp_window_size",\
#                 "tcp_checksum",\
#                 "tcp_srcport",\
#                 "tcp_dstport",\
#                 "label"

# "frame_info_time"
# selected after feature importance with fake attributes
probeDf = probeDf.select(*("frame_info_len", "tcp_ack",\
                "tcp_seq",\
                "ip_len", "tcp_flags",\
                "tcp_options_mss_val",\
                "ip_ttl",\
                "tcp_window_size",\
                "tcp_checksum",\
                "tcp_srcport",\
                "tcp_dstport",\
                "label"))


In [ ]:
%%python
probeDf.printSchema()

In [12]:
%%python
from pyspark.sql.functions import col
for col_ in probeDf.columns[:len(probeDf.columns)-1]:
    if col_ == "tcp_options_mss_val":
        probeDf = probeDf.withColumn(col_, col(col_).cast("float"))
    else:
        probeDf = probeDf.withColumn(col_, col(col_).cast("integer"))
probeDf.printSchema()

root
 |-- frame_info_len: integer (nullable = true)
 |-- tcp_ack: integer (nullable = true)
 |-- tcp_seq: integer (nullable = true)
 |-- ip_len: integer (nullable = true)
 |-- tcp_flags: integer (nullable = true)
 |-- tcp_options_mss_val: float (nullable = true)
 |-- ip_ttl: integer (nullable = true)
 |-- tcp_window_size: integer (nullable = true)
 |-- tcp_checksum: integer (nullable = true)
 |-- tcp_srcport: integer (nullable = true)
 |-- tcp_dstport: integer (nullable = true)
 |-- label: string (nullable = true)



In [13]:
%%python
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(probeDf)

In [14]:
%%python
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["frame_info_len", "tcp_ack",\
                "tcp_seq",\
                "ip_len", "tcp_flags",\
                "tcp_options_mss_val",\
                "ip_ttl",\
                "tcp_window_size",\
                "tcp_checksum",\
                "tcp_srcport",\
                "tcp_dstport"],
    outputCol="features")

output = assembler.setHandleInvalid("skip").transform(probeDf)

In [15]:
%%python
probeDf = output.select(*("features","label"))
probeDf.show()

+--------------------+------+
|            features| label|
+--------------------+------+
|[54.0,0.0,0.0,40....|normal|
|[551.0,1.0,1.0,53...|normal|
|[94.0,1.0,1.0,80....|normal|
|[68.0,1.0,1.0,54....|normal|
|[54.0,0.0,0.0,40....|normal|
|[54.0,0.0,0.0,40....|normal|
|[1440.0,1.0,1.0,1...|normal|
|[1440.0,1.0,2749....|normal|
|[1440.0,1.0,1.0,1...|normal|
|[1440.0,1.0,31603...|normal|
|[54.0,0.0,0.0,40....|normal|
|[1514.0,1.0,1.0,1...|normal|
|[1514.0,1.0,1.0,1...|normal|
|[1514.0,1.0,23714...|normal|
|[1514.0,1.0,32474...|normal|
|[2974.0,1.0,37207...|normal|
|[1470.0,1.0,1.0,1...|normal|
|[1514.0,1.0,94099...|normal|
|[54.0,0.0,0.0,40....|normal|
|[1514.0,1.0,19403...|normal|
+--------------------+------+
only showing top 20 rows



In [16]:
%%python
from pyspark.ml.feature import VectorIndexer
featureIndexer =\
    VectorIndexer(inputCol="features",\
    outputCol="indexedFeatures",\
    maxCategories=3).fit(probeDf)

In [16]:
%%python
DF = probeDf.toPandas()

In [17]:
%%python
labelTypes = ['normal', 'nmap_null', 'nmap_connect', 'zmap', 'nmap_window', 'masscan', 'hping_syn', 'unicorn_null', 'unicorn_syn', 'nmap_xmas', 'nmap_syn', 'unicorn_conn', 'unicorn_xmas', 'nmap_ack', 'hping_fin', 'nmap_maimon', 'hping_null', 'hping_xmas', 'hping_ack', 'nmap_fin', 'unicorn_fxmas']

In [18]:
%%python
import pandas as pd
df = pd.read_csv("newPortData.csv")

indices = dict()
for col in labelTypes:
    indices[col] = list(df[df['label'] == col]["Unnamed: 0"])

In [20]:
%%python
L = []
L += [2]
L

[2]

In [24]:
/*!pip3 install imblearn*/

_: Any = [2]
_20: Any = [2]


In [29]:
%%python
DF.head()

,features,label
0,"[54.0, 0.0, 0.0, 40.0, 2.0, 17055.138671875, 2...",normal
1,"[551.0, 1.0, 1.0, 537.0, 24.0, 16923.056640625...",normal
2,"[94.0, 1.0, 1.0, 80.0, 16.0, 16738.109375, 59....",normal
3,"[68.0, 1.0, 1.0, 54.0, 24.0, 16507.53515625, 5...",normal
4,"[54.0, 0.0, 0.0, 40.0, 2.0, 16237.595703125, 2...",normal


In [74]:
%%python
probeDf.sample(False, 0.5, seed=0).show()

+--------------------+------+
|            features| label|
+--------------------+------+
|[94.0,1.0,1.0,80....|normal|
|[68.0,1.0,1.0,54....|normal|
|[54.0,0.0,0.0,40....|normal|
|[1440.0,1.0,1.0,1...|normal|
|[1440.0,1.0,31603...|normal|
|[54.0,0.0,0.0,40....|normal|
|[1514.0,1.0,1.0,1...|normal|
|[1514.0,1.0,32474...|normal|
|[2974.0,1.0,37207...|normal|
|[1470.0,1.0,1.0,1...|normal|
|[1514.0,1.0,94099...|normal|
|[1514.0,1.0,19403...|normal|
|[1514.0,1.0,31476...|normal|
|[1514.0,1.0,32060...|normal|
|[1514.0,1.0,32388...|normal|
|[117.0,1.0,1.0,10...|normal|
|[1514.0,1.0,17181...|normal|
|[1440.0,1.0,11129...|normal|
|[1514.0,1.0,49531...|normal|
|[180.0,1.0,1.0,16...|normal|
+--------------------+------+
only showing top 20 rows



In [18]:
%%python
# doing data split

newTest = None
newTrain = None

for col in labelTypes:
    allCol = probeDf.filter(probeDf["label"]==col)
    newColFiltered = allCol.sample(False, 0.5, seed=101)
    if newTest is None:
        newTest = (allCol).subtract(newColFiltered)
    else:
        newTest = newTest.union((allCol).subtract(newColFiltered))
    
    if newTrain is None:
        newTrain = newColFiltered
    else:
        newTrain = newTrain.union(newColFiltered)



In [19]:
%%python
from pyspark.ml.classification import RandomForestClassifier
#(trainingData, testData) = probeDf.randomSplit([0.70, 0.30])

rfProbe = RandomForestClassifier(labelCol="indexedLabel", 
                                 featuresCol="indexedFeatures", 
                                 numTrees = 5,
                                 maxDepth = 3)


In [20]:
%%python
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rfProbe])

In [21]:
%%python
newTrain.show()

+--------------------+------+
|            features| label|
+--------------------+------+
|[54.0,0.0,0.0,40....|normal|
|[94.0,1.0,1.0,80....|normal|
|[1440.0,1.0,1.0,1...|normal|
|[1440.0,1.0,2749....|normal|
|[1440.0,1.0,1.0,1...|normal|
|[54.0,0.0,0.0,40....|normal|
|[1514.0,1.0,1.0,1...|normal|
|[1470.0,1.0,1.0,1...|normal|
|[1514.0,1.0,28048...|normal|
|[1514.0,1.0,32388...|normal|
|[117.0,1.0,1.0,10...|normal|
|[1440.0,1.0,70075...|normal|
|[54.0,0.0,0.0,40....|normal|
|[1440.0,1.0,93433...|normal|
|[54.0,0.0,0.0,40....|normal|
|[1514.0,1.0,17181...|normal|
|[1440.0,1.0,11129...|normal|
|[1514.0,1.0,21368...|normal|
|[180.0,1.0,1.0,16...|normal|
|[1514.0,1.0,22722...|normal|
+--------------------+------+
only showing top 20 rows



In [ ]:
%%python
model = pipeline.fit(newTrain)

In [ ]:
%%python
predictions = model.transform(newTest)

In [ ]:
%%python
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

In [60]:
%%python
rfModel = model.stages[2]
print(rfModel.featureImportances)

(11,[0,1,2,3,4,5,6,7,8,9,10],[0.10225270767747473,0.04275038269299825,0.028624979124243746,0.014479264860995038,0.08524591797888197,0.02793738248619991,0.3552375167831205,0.12485313758240703,0.003330396690295598,0.059554574006360626,0.15573374011702254])
